In [1]:
#################################################
### Assignment 5                              ###
### Dvir David Iluz                           ###
### 311447668                                 ###
#################################################

### Import libraries and assign API

In [2]:
import json
import requests
import pandas as pd
import googlemaps as gm
from bs4 import BeautifulSoup

### Read csv file & save api

In [3]:
api = 'fakeapiforgooglecloud'
file = pd.read_csv("destinations_LP_crawler_Ex5.csv")

### Function to find country by lat and lng values

In [4]:
def reverseGeocode(cords):
    gmaps = gm.Client(key=api)
    res = gmaps.reverse_geocode(cords)
    country = res[-1]['address_components'][0]['long_name']
    return country

### Getting all cities lat and lng values calling the reverseGeo function to append countries to a list

In [5]:
countries_list = []
for city in file['city']:
    url_geo = 'https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s' % (city,api)
    try:
        response_geo = requests.get(url_geo)
        if not response_geo.status_code == 200:
            print('HTTP ERROR', response.status_code)
        else:
            try:
                response_data_geo = response_geo.json()
            except:
                print("response not in valid JSON format")
    except:
        print("Something went wrong with requests.get")
    location_lat = str(response_data_geo['results'][0]['geometry']['location']['lat'])
    location_lng = str(response_data_geo['results'][0]['geometry']['location']['lng'])
    cord = (location_lat,location_lng)
    country = reverseGeocode(cord)
    countries_list.append(country)

### Create URLs & Fix United States to usa

In [6]:
url_base = 'https://www.lonelyplanet.com/'
urls = []
for index,extension in enumerate(file['city_LP']):
    url = url_base+countries_list[index]+"/"+extension
    urls.append(url)
urls = [word.replace('United States', 'usa') for word in urls]

### Get description for every url and append it to a list

In [16]:
descriptions = []
for url in urls:
    response = requests.get(url)
    if not response.status_code == 200:
        print(f"Error, {url}")
        descriptions.append(' ')
    else: 
        print(f'Success, {url}')
    result = BeautifulSoup(response.content,'html.parser')
    for descr in result.find_all('div',class_='readMore_content___5EuR relative overflow-hidden max-h-96 is-max wysiwyg'):
        descriptions.append(descr.get_text())

Success, https://www.lonelyplanet.com/Turkey/mediterranean-coast/antalya
Success, https://www.lonelyplanet.com/Greece/Athens
Success, https://www.lonelyplanet.com/Cyprus/the-republic-of-cyprus/Agia-napa
Success, https://www.lonelyplanet.com/Thailand/Bangkok
Success, https://www.lonelyplanet.com/Spain/Barcelona
Success, https://www.lonelyplanet.com/Switzerland/northern-switzerland/basel
Success, https://www.lonelyplanet.com/Georgia/adjara/batumi
Success, https://www.lonelyplanet.com/China/Beijing
Success, https://www.lonelyplanet.com/Germany/Berlin
Success, https://www.lonelyplanet.com/usa/Boston
Success, https://www.lonelyplanet.com/Slovakia/Bratislava
Success, https://www.lonelyplanet.com/Belgium/Brussels
Success, https://www.lonelyplanet.com/Romania/Bucharest
Success, https://www.lonelyplanet.com/Hungary/Budapest
Success, https://www.lonelyplanet.com/Bulgaria/black-sea-coast/burgas
Success, https://www.lonelyplanet.com/Mexico/Cancun
Success, https://www.lonelyplanet.com/Italy/sicily/

#### As you can see we received an error for Israel, because google give us Israel, 
#### and the website real address is https://www.lonelyplanet.com/israel-and-the-palestinian-territories/the-negev/eilat
#### israel-and-the-palestinian-territories
#### This is something I wont change,
#### because it goes against my beliefs and principles.

### Create dataframe with city, country and description and make csv file

In [17]:
dict_all = {'City': file['city'], 'Country': countries_list, 'Description': descriptions}
df = pd.DataFrame(dict_all)
df.to_csv('LP_destinations.csv')

### Presenting the DataFrame

In [18]:
df

,City,Country,Description
0,Antalya,Turkey,Once seen simply as the gateway to the Turkish...
1,Athens,Greece,"With equal measures of grunge and grace, Athen..."
2,Ayia-napa,Cyprus,Endless blue-sky days and a coastline riddled ...
3,Bangkok,Thailand,"Same same, but different. This Thailish T-shir..."
4,Barcelona,Spain,Barcelona is an enchanting seaside city with b...
...,...,...,...
72,Venice,Italy,Imagine the audacity of building a city of mar...
73,Vienna,Austria,Baroque streetscapes and imperial palaces set ...
74,Warsaw,Poland,"A phoenix arisen from the ashes, Poland's capi..."
75,Zakynthos,Greece,"Zakynthos, also known by its Italian name Zant..."
